#import, 설치, 구글 드라이브 마운트

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import os

import torch
import torch.nn as nn
from tqdm import trange
from transformers import XLMRobertaModel, AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from datasets import load_metric
from sklearn.metrics import f1_score
import pandas as pd
import copy

import json
import pandas as pd

from collections import Counter

from random import randint 

json 및 jsonl 파일 read, write 함수

In [ ]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j


# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

# Auto Ensemble 사전 준비
load model

이름, 모델 개수, 저장 장소, 실행 횟수

## dataframe 리스트를 수정

In [ ]:
model_A = pd.DataFrame(jsonlload("/content/model_A.jsonl"))
model_B = pd.DataFrame(jsonlload("/content/model_B.jsonl"))
model_C = pd.DataFrame(jsonlload("/content/model_C.jsonl"))
model_D = pd.DataFrame(jsonlload("/content/model_D.jsonl"))
model_E = pd.DataFrame(jsonlload("/content/model_E.jsonl"))

In [ ]:
ensemble_models = [model_A, model_B, model_C, model_D, model_E]


# 위 모델 리스트 내용을 복사해서 ''' (여기) ''' 에 복사 붙여넣기
ensemble_name = """[model_A, model_B, model_C, model_D, model_E]"""


ensemble_name = ensemble_name.replace("[", "").replace("]", "").replace(" ", "").replace("\n", "")
ensemble_name = ensemble_name.split(",")


len(ensemble_models), len(ensemble_name)


In [ ]:
# 이름을 입력하세요
name = "Auto_Ensemble"

# 몇 가지 모델을 앙상블 할까요?
# 중복이 포함될 수 있습니다.
num_of_ensemble_model = 15

# 저장 장소  (마지막에 /도 써주세요)
save_path = "/content/"

# 오토앙상블을 몇 번 실행할까요?
excute_num = 10

In [ ]:
# 기록 파일 생성
with open(save_path + name + ".txt", "w") as file:
    pass

# function

## (func) 오토 앙상블 모델조합 + 기록 

In [ ]:
def model_combination_and_recode(num):
    automatic_ae = []
    model_names = []

    for i in range(num_of_ensemble_model):
        idx = randint( 0, len(ensemble_models)-1 )
        model = ensemble_models[ idx ]

        automatic_ae.append( model )
        model_names.append( ensemble_name[idx] )
    
    with open(save_path + name + ".txt", "r+") as file:
        while True:
            s = file.readline()
            if not s: 
                break
        
        file.write(name)
        file.write(str(num))
        file.write("=")
        file.write(str(model_names).replace("\'", ""))
        file.write("\n")
    print(name,"_", num, " = ", str(model_names))

    return automatic_ae


## (func) 오토 앙상블 실행

In [ ]:
def excute_ensemble(automatic_ae):
    dic_ae = {
        'id' : [],
        'sentence_form' : [],
        'annotation' : []
    }

    models = automatic_ae 


    for i in range(len(models[0])):
        tmp_divide = []

        for j in models:
            tmp_divide.append( str(j['annotation'][i]) )

        answer = Counter(tmp_divide).most_common(n=1)[0][0]

        check =  str(tmp_divide).replace("[", "").replace("]", "").replace(",", "").replace("\'", "").replace("\"", "").replace(" ", "")

        if '[]' in answer and check is not "":
            while '[]' in tmp_divide:
                tmp_divide.remove('[]')
            dic_ae['annotation'].append( tmp_divide[0] )
        else:
            dic_ae['annotation'].append( answer )
        dic_ae['id'].append( j['id'][i] )
        dic_ae['sentence_form'].append( j['sentence_form'][i] )

    df_ae = pd.DataFrame( dic_ae )

    return df_ae

## (func) jsonl 파일로 저장

In [ ]:
def save_jsonl(df_ae, num):

    with open(save_path + "{0}_{1}.jsonl" .format(name, num), 'w') as file:
        for i in range( len(df_ae) ):
            annos = df_ae['annotation'][i]
            str_annos = str(annos)
            tmp = str_annos.replace("None", "null").replace("\'", "\"")

            file.write(  '{'+'\"id\": \"nikluge-sa-2022-{3}-{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                        .format( str(i+1).zfill(5)  ,   df_ae['sentence_form'][i], tmp, name ) +'}' )
            file.write("\n")

#{"id": "nikluge-sa-2022-dev-00001", "sentence_form": "깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면 금방 우러난다.", "annotation": [["본품#편의성", ["부직포 포장", 5, 11], "positive"]]}
# 0은 숫자, 1은 sentence, 2는 annotation, 3은 file_name(파일이름)

# 앙상블 시작

In [ ]:
start_num = 1


#####

for i in range( start_num,  excute_num + start_num) :
    models = model_combination_and_recode(i)
    ensemble_model = excute_ensemble(models)
    save_jsonl(ensemble_model, i)

## 잘 열리는지 test

In [ ]:
file_name = 'Auto_Ensemble_1'

df_test =  pd.DataFrame(jsonlload("/content/{0}.jsonl". format(file_name)))
df_test